In [1]:
from pandas import DataFrame, read_csv, concat
import random
import math
from useGeneratedData import splitTrainTest, NewPredicter, makeDataUsable

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Testing data
size = 20
paramsToStore = {"Time": [0,1,2,3,0,1,2,3,4,5,0,1,2,3,4,5,6,7,8,9],
                "TTC": [random.uniform(0, 10) for _ in range(size)],
                "DTO": [random.uniform(0, 20) for _ in range(size)],
                "JERK": [abs(random.gauss(0, 20)) for _ in range(size)],
                "Speed": [random.uniform(0, 15) for _ in range(size)],
                "asX": [random.gauss(0, 0.15) for _ in range(size)],
                "asY": [random.gauss(0, 0.15) for _ in range(size)],
                "asZ": [random.gauss(0, 0.1) for _ in range(size)],
                "COL": [0,0,0,1,0,0,0,1,0,0, 0,0,0,1,0,0,0,1,0,0]}
df = DataFrame(paramsToStore)
# df.set_index("Time", inplace=True)
# df.to_csv(PATH+"/data/testGenerateData.csv", index=False)
df.head(20)

,Time,TTC,DTO,JERK,Speed,asX,asY,asZ,COL
0,0,1.494333,9.362956,39.442354,12.414565,0.044294,0.063726,0.134401,0
1,1,0.552172,18.203646,2.736524,10.609841,-0.129643,0.113939,0.163005,0
2,2,8.685209,14.006805,5.956925,12.416173,0.165349,-0.185235,-0.154655,0
3,3,2.564148,7.308861,28.629364,12.992678,0.194372,-0.094533,-0.016750,1
4,0,4.350996,1.073657,9.586924,14.750311,0.131293,-0.217560,0.082002,0
5,1,9.381818,14.555515,16.749348,14.631894,-0.133235,-0.400881,0.063552,0
6,2,5.476967,5.412819,9.428627,9.436979,0.349184,-0.059383,0.029775,0
7,3,0.627454,17.352438,27.616409,6.129018,-0.054881,-0.099159,-0.040610,1
8,4,2.962535,0.021050,10.538351,4.802034,0.016591,0.048284,0.054208,0
9,5,6.991022,1.430222,12.102980,4.879655,0.161041,0.160197,0.047723,0


In [3]:
def makeDataUsable(df: DataFrame, pastImportance: int=6, rowsBeforeCol: int=3, removeCol: bool=True) -> DataFrame:
    """
    Make a usable Dataframe from a csv file from generated data.\\
    Creates pastImportance of columns for each prediction feature.\\
    Registers the "col" feature before an actual collision as 1, up to secBeforeCol back in time.\\
    If undersampleRatio is > 0, it will under sample the data, if it is 0, it will not.
    
    ### Params:
        * df: Dataframe, need to have the columns named "TTC", "DTO", "JERK" and "Speed"ArithmeticError
        * pastImportance: int, needs to be > 0
        * rowsBeforeCol: int, needs to be > 0
        * removeCol: bool, True if the origianl collision row should be removed
    ### Returns:
        * Dataframe
    """
    colsToUse = ["TTC", "DTO", "JERK", "Speed", "asX", "asY", "asZ"]
    columns = ["Time"]
    columns += [f"{c}{i}" for c in colsToUse for i in range(1, pastImportance+1)]
    columns += ["COL", "toRemove"]

    rowsToRemove = []
    
    dataDict = {col: [] for col in columns}

    for i, row in df.iterrows():
        if row["Time"] < pastImportance-1:
            continue

        dataDict["Time"].append(row["Time"])
        for j, k in enumerate(range(i-pastImportance+1, i+1), start=1):
            for c in colsToUse:
                # Should be able to remove this if check with new data
                if (c == "TTC" or c == "DTO") and df.iloc[k][c] < 0:
                    dataDict[f"{c}{j}"].append(0)
                    continue
                dataDict[f"{c}{j}"].append(df.iloc[k][c])

        dataDict["COL"].append(row["COL"])
        dataDict["toRemove"].append(row["COL"])
        if row["COL"] == 1:
            rowsToRemove.append(i-1)
            available = int(row["Time"]-pastImportance+2)
            amount = int(rowsBeforeCol) if available >= rowsBeforeCol else available
            dataDict["COL"][-amount:] = [1]*amount

    df = DataFrame(dataDict)
    df = df[df["toRemove"] == 0] if removeCol else df
    df.drop("toRemove", axis=1, inplace=True)
    return df

# makeDataUsable(read_csv("../../data/testGenerateData.csv"), pastImportance=4, secBeforeCol=4)

In [4]:
makeDataUsable(df, 2, 3, True)

,Time,TTC1,TTC2,DTO1,DTO2,JERK1,JERK2,Speed1,Speed2,asX1,asX2,asY1,asY2,asZ1,asZ2,COL
0,1.0,1.494333,0.552172,9.362956,18.203646,39.442354,2.736524,12.414565,10.609841,0.044294,-0.129643,0.063726,0.113939,0.134401,0.163005,1.0
1,2.0,0.552172,8.685209,18.203646,14.006805,2.736524,5.956925,10.609841,12.416173,-0.129643,0.165349,0.113939,-0.185235,0.163005,-0.154655,1.0
3,1.0,4.350996,9.381818,1.073657,14.555515,9.586924,16.749348,14.750311,14.631894,0.131293,-0.133235,-0.217560,-0.400881,0.082002,0.063552,1.0
4,2.0,9.381818,5.476967,14.555515,5.412819,16.749348,9.428627,14.631894,9.436979,-0.133235,0.349184,-0.400881,-0.059383,0.063552,0.029775,1.0
6,4.0,0.627454,2.962535,17.352438,0.021050,27.616409,10.538351,6.129018,4.802034,-0.054881,0.016591,-0.099159,0.048284,-0.040610,0.054208,0.0
7,5.0,2.962535,6.991022,0.021050,1.430222,10.538351,12.102980,4.802034,4.879655,0.016591,0.161041,0.048284,0.160197,0.054208,0.047723,0.0
8,1.0,0.853033,6.909341,17.806971,8.082571,28.962532,24.246312,10.330287,12.801631,0.003159,0.237234,-0.269593,0.062156,0.106314,-0.050474,1.0
9,2.0,6.909341,1.996912,8.082571,1.569399,24.246312,7.455405,12.801631,11.515686,0.237234,-0.318824,0.062156,0.034181,-0.050474,0.019541,1.0
11,4.0,7.202963,8.681606,17.652759,17.119338,7.169374,12.533777,12.081950,14.036239,0.138961,0.202392,0.122379,0.154962,-0.079951,0.072961,0.0
12,5.0,8.681606,5.646413,17.119338,5.927950,12.533777,26.355277,14.036239,2.066720,0.202392,0.199665,0.154962,-0.121723,0.072961,0.111459,1.0


In [5]:
rawData = read_csv("../../data/generatedData.csv")
rawData.iloc[2401:2410]

,Time,TTC,DTO,JERK,Speed,asX,asY,asZ,COL
2401,12.0,13.438,20.4665,0.196,16.427,0.000,0.00,-0.000,0
2402,12.5,15.222,19.8645,0.008,16.226,0.000,0.00,-0.000,0
2403,13.0,15.046,19.2738,0.008,16.027,-0.000,0.00,-0.000,0
2404,13.5,19.378,18.8359,0.028,15.835,0.000,0.00,0.000,0
2405,14.0,1.174,13.2444,0.008,15.645,-0.000,0.00,0.000,0
2406,14.5,0.350,5.4750,0.016,15.451,-0.000,0.01,0.000,0
2407,15.0,0.053,0.8400,52.528,2.125,0.116,-0.10,0.065,1
2408,15.5,50.000,2.0090,52.320,1.879,-0.040,-0.00,-0.019,1
2409,16.0,50.000,3.3947,0.920,1.863,-0.000,-0.00,-0.000,0


In [6]:
len(rawData)//2/60

34.81666666666667

In [7]:
data = makeDataUsable(read_csv("../../data/generatedData.csv"), pastImportance=4, rowsBeforeCol=5, removeCol=True)
data.iloc[2401:2410]


,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,...,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
2472,54.5,0.446,0.081,50.000,50.000,5.4459,1.4735,3.8853,5.3391,0.024,...,-0.002,0.000,0.018,-0.000,-0.000,0.000,0.052,0.001,0.002,0.0
2473,55.0,0.081,50.000,50.000,50.000,1.4735,3.8853,5.3391,6.0938,81.132,...,-0.002,0.018,-0.000,-0.000,0.045,0.052,0.001,0.002,-0.002,0.0
2474,55.5,50.000,50.000,50.000,50.000,3.8853,5.3391,6.0938,6.2336,81.992,...,-0.003,-0.000,-0.000,0.045,-0.173,0.001,0.002,-0.002,-0.002,0.0
2475,56.0,50.000,50.000,50.000,50.000,5.3391,6.0938,6.2336,100.0000,4.552,...,-0.003,-0.000,0.045,-0.173,-0.342,0.002,-0.002,-0.002,0.003,0.0
2476,56.5,50.000,50.000,50.000,50.000,6.0938,6.2336,100.0000,79.7880,0.448,...,0.003,0.045,-0.173,-0.342,-0.006,-0.002,-0.002,0.003,0.006,0.0
2477,57.0,50.000,50.000,50.000,36.513,6.2336,100.0000,79.7880,78.0648,6.736,...,0.021,-0.173,-0.342,-0.006,0.151,-0.002,0.003,0.006,0.024,0.0
2478,57.5,50.000,50.000,36.513,50.000,100.0000,79.7880,78.0648,100.0000,5.816,...,0.028,-0.342,-0.006,0.151,0.231,0.003,0.006,0.024,0.019,0.0
2479,58.0,50.000,36.513,50.000,50.000,79.7880,78.0648,100.0000,19.7160,2.400,...,0.072,-0.006,0.151,0.231,1.258,0.006,0.024,0.019,0.035,0.0
2480,58.5,36.513,50.000,50.000,1.624,78.0648,100.0000,19.7160,15.1854,0.080,...,-0.073,0.151,0.231,1.258,0.674,0.024,0.019,0.035,0.004,0.0


In [34]:
cols= data.columns
s = 9
s1 = 19
display(data[cols[:s]].iloc[2323:2331])
display(data[cols[s:s1]].iloc[2323:2331])
display(data[cols[s1:]].iloc[2323:2331])

,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4
2386,11.5,2.932,6.499,22.546,11.917,24.3219,22.6356,22.0272,21.1773
2387,12.0,6.499,22.546,11.917,13.438,22.6356,22.0272,21.1773,20.4665
2388,12.5,22.546,11.917,13.438,15.222,22.0272,21.1773,20.4665,19.8645
2389,13.0,11.917,13.438,15.222,15.046,21.1773,20.4665,19.8645,19.2738
2390,13.5,13.438,15.222,15.046,19.378,20.4665,19.8645,19.2738,18.8359
2391,14.0,15.222,15.046,19.378,1.174,19.8645,19.2738,18.8359,13.2444
2392,14.5,15.046,19.378,1.174,0.350,19.2738,18.8359,13.2444,5.4750
2395,16.0,0.350,0.053,50.000,50.000,5.4750,0.8400,2.0090,3.3947


,JERK1,JERK2,JERK3,JERK4,Speed1,Speed2,Speed3,Speed4,asX1,asX2
2386,0.072,0.048,2.800,2.536,16.524,16.304,16.784,16.630,-0.007,-0.033
2387,0.048,2.800,2.536,0.196,16.304,16.784,16.630,16.427,-0.033,0.016
2388,2.800,2.536,0.196,0.008,16.784,16.630,16.427,16.226,0.016,0.001
2389,2.536,0.196,0.008,0.008,16.630,16.427,16.226,16.027,0.001,0.000
2390,0.196,0.008,0.008,0.028,16.427,16.226,16.027,15.835,0.000,0.000
2391,0.008,0.008,0.028,0.008,16.226,16.027,15.835,15.645,0.000,-0.000
2392,0.008,0.028,0.008,0.016,16.027,15.835,15.645,15.451,-0.000,0.000
2395,0.016,52.528,52.320,0.920,15.451,2.125,1.879,1.863,-0.000,0.116


,asX3,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
2386,0.016,0.001,0.002,-0.235,-0.004,0.00,0.007,0.034,-0.001,-0.003,0.0
2387,0.001,0.000,-0.235,-0.004,0.000,0.00,0.034,-0.001,-0.003,-0.000,0.0
2388,0.000,0.000,-0.004,0.000,0.000,0.00,-0.001,-0.003,-0.000,-0.000,0.0
2389,0.000,-0.000,0.000,0.000,0.000,0.00,-0.003,-0.000,-0.000,-0.000,1.0
2390,-0.000,0.000,0.000,0.000,0.000,0.00,-0.000,-0.000,-0.000,0.000,1.0
2391,0.000,-0.000,0.000,0.000,0.000,0.00,-0.000,-0.000,0.000,0.000,1.0
2392,-0.000,-0.000,0.000,0.000,0.000,0.01,-0.000,0.000,0.000,0.000,1.0
2395,-0.040,-0.000,0.010,-0.100,-0.000,-0.00,0.000,0.065,-0.019,-0.000,0.0


In [44]:
cols= data.columns
c = ['Time', 'TTC1', 'TTC2', 'TTC3', 'TTC4', 'DTO1', 'DTO2', 'DTO3', 'DTO4',
       'JERK1', 'JERK2', 'JERK3', 'JERK4', 'Speed1', 'Speed2', 'Speed3',
       'Speed4', 'asX1', 'asX2', 'asX3', 'asX4', 'asY1', 'asY2', 'asY3',
       'asY4', 'asZ1', 'asZ2', 'asZ3', 'asZ4', 'COL'],

display(data[['Time', 'TTC1', 'TTC2', 'TTC3', 'TTC4', 'DTO1', 'DTO2', 'DTO3', 'DTO4','JERK1', 'JERK2', 'JERK3']].iloc[2325:2331])
display(data[["Time", 'JERK4', 'Speed1', 'Speed2', 'Speed3','Speed4', 'asX1', 'asX2', 'asX3', 'asX4', 'asY1','asY2', 'asY3']].iloc[2325:2331])
display(data[['Time','asX1', 'asX2', 'asX3', 'asX4', 'asY1', 'asY2', 'asY3','asY4', 'asZ1', 'asZ2', 'asZ3', 'asZ4', 'COL']].iloc[2325:2331])

,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,JERK2,JERK3
2388,12.5,22.546,11.917,13.438,15.222,22.0272,21.1773,20.4665,19.8645,2.800,2.536,0.196
2389,13.0,11.917,13.438,15.222,15.046,21.1773,20.4665,19.8645,19.2738,2.536,0.196,0.008
2390,13.5,13.438,15.222,15.046,19.378,20.4665,19.8645,19.2738,18.8359,0.196,0.008,0.008
2391,14.0,15.222,15.046,19.378,1.174,19.8645,19.2738,18.8359,13.2444,0.008,0.008,0.028
2392,14.5,15.046,19.378,1.174,0.350,19.2738,18.8359,13.2444,5.4750,0.008,0.028,0.008
2395,16.0,0.350,0.053,50.000,50.000,5.4750,0.8400,2.0090,3.3947,0.016,52.528,52.320


,Time,JERK4,Speed1,Speed2,Speed3,Speed4,asX1,asX2,asX3,asX4,asY1,asY2,asY3
2388,12.5,0.008,16.784,16.630,16.427,16.226,0.016,0.001,0.00,0.0,-0.004,0.0,0.0
2389,13.0,0.008,16.630,16.427,16.226,16.027,0.001,0.000,0.00,-0.0,0.000,0.0,0.0
2390,13.5,0.028,16.427,16.226,16.027,15.835,0.000,0.000,-0.00,0.0,0.000,0.0,0.0
2391,14.0,0.008,16.226,16.027,15.835,15.645,0.000,-0.000,0.00,-0.0,0.000,0.0,0.0
2392,14.5,0.016,16.027,15.835,15.645,15.451,-0.000,0.000,-0.00,-0.0,0.000,0.0,0.0
2395,16.0,0.920,15.451,2.125,1.879,1.863,-0.000,0.116,-0.04,-0.0,0.010,-0.1,-0.0


,Time,asX1,asX2,asX3,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
2388,12.5,0.016,0.001,0.00,0.0,-0.004,0.0,0.0,0.00,-0.001,-0.003,-0.000,-0.0,0.0
2389,13.0,0.001,0.000,0.00,-0.0,0.000,0.0,0.0,0.00,-0.003,-0.000,-0.000,-0.0,1.0
2390,13.5,0.000,0.000,-0.00,0.0,0.000,0.0,0.0,0.00,-0.000,-0.000,-0.000,0.0,1.0
2391,14.0,0.000,-0.000,0.00,-0.0,0.000,0.0,0.0,0.00,-0.000,-0.000,0.000,0.0,1.0
2392,14.5,-0.000,0.000,-0.00,-0.0,0.000,0.0,0.0,0.01,-0.000,0.000,0.000,0.0,1.0
2395,16.0,-0.000,0.116,-0.04,-0.0,0.010,-0.1,-0.0,-0.00,0.000,0.065,-0.019,-0.0,0.0


In [33]:
cols= data.columns
s = 14
display(data[cols[:s]].iloc[2323:2331])
display(data[cols[s:]].iloc[2323:2331])

,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,JERK2,JERK3,JERK4,Speed1
2386,11.5,2.932,6.499,22.546,11.917,24.3219,22.6356,22.0272,21.1773,0.072,0.048,2.800,2.536,16.524
2387,12.0,6.499,22.546,11.917,13.438,22.6356,22.0272,21.1773,20.4665,0.048,2.800,2.536,0.196,16.304
2388,12.5,22.546,11.917,13.438,15.222,22.0272,21.1773,20.4665,19.8645,2.800,2.536,0.196,0.008,16.784
2389,13.0,11.917,13.438,15.222,15.046,21.1773,20.4665,19.8645,19.2738,2.536,0.196,0.008,0.008,16.630
2390,13.5,13.438,15.222,15.046,19.378,20.4665,19.8645,19.2738,18.8359,0.196,0.008,0.008,0.028,16.427
2391,14.0,15.222,15.046,19.378,1.174,19.8645,19.2738,18.8359,13.2444,0.008,0.008,0.028,0.008,16.226
2392,14.5,15.046,19.378,1.174,0.350,19.2738,18.8359,13.2444,5.4750,0.008,0.028,0.008,0.016,16.027
2395,16.0,0.350,0.053,50.000,50.000,5.4750,0.8400,2.0090,3.3947,0.016,52.528,52.320,0.920,15.451


,Speed2,Speed3,Speed4,asX1,asX2,asX3,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
2386,16.304,16.784,16.630,-0.007,-0.033,0.016,0.001,0.002,-0.235,-0.004,0.00,0.007,0.034,-0.001,-0.003,0.0
2387,16.784,16.630,16.427,-0.033,0.016,0.001,0.000,-0.235,-0.004,0.000,0.00,0.034,-0.001,-0.003,-0.000,0.0
2388,16.630,16.427,16.226,0.016,0.001,0.000,0.000,-0.004,0.000,0.000,0.00,-0.001,-0.003,-0.000,-0.000,0.0
2389,16.427,16.226,16.027,0.001,0.000,0.000,-0.000,0.000,0.000,0.000,0.00,-0.003,-0.000,-0.000,-0.000,1.0
2390,16.226,16.027,15.835,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.00,-0.000,-0.000,-0.000,0.000,1.0
2391,16.027,15.835,15.645,0.000,-0.000,0.000,-0.000,0.000,0.000,0.000,0.00,-0.000,-0.000,0.000,0.000,1.0
2392,15.835,15.645,15.451,-0.000,0.000,-0.000,-0.000,0.000,0.000,0.000,0.01,-0.000,0.000,0.000,0.000,1.0
2395,2.125,1.879,1.863,-0.000,0.116,-0.040,-0.000,0.010,-0.100,-0.000,-0.00,0.000,0.065,-0.019,-0.000,0.0


In [9]:
cols:s

In [10]:
cc = read_csv("../../data/generatedData.csv")
collisions = len(data[data["COL"] == 1])

print(f"Amount of unique collisions: {len(cc[cc['COL'] == 1])}")
print(f"Amount of collisions: {len(data[data['COL'] == 1])}")
print(f"Amount not colliding: {len(data[data['COL'] == 0])}")

Amount of unique collisions: 215
Amount of collisions: 492
Amount not colliding: 3458


In [13]:
data.corr()["COL"]

Time     -0.065969
TTC1     -0.101218
TTC2     -0.177124
TTC3     -0.251498
TTC4     -0.316259
DTO1     -0.062557
DTO2     -0.109763
DTO3     -0.157293
DTO4     -0.208628
JERK1    -0.051574
JERK2    -0.043444
JERK3    -0.045155
JERK4    -0.068430
Speed1    0.154619
Speed2    0.170875
Speed3    0.174013
Speed4    0.168772
asX1     -0.010917
asX2     -0.007154
asX3     -0.014407
asX4     -0.017245
asY1      0.017051
asY2      0.018624
asY3      0.000970
asY4     -0.001754
asZ1      0.014642
asZ2      0.021415
asZ3      0.024463
asZ4      0.038726
COL       1.000000
Name: COL, dtype: float64

In [97]:
def getResultsFromData(filename: str, pastImportance: int=4, rowsBeforeCol: int=4, removeCol: bool=True, undersampleRatio: float=1):
    """
    TODO
    ### Params:
        * undersampleRatio: float, ratio of non collisions / collisions
    """
    data = makeDataUsable(read_csv(filename), pastImportance, rowsBeforeCol, removeCol)
    # dataToUse = data.copy()
    data.drop(columns=data.columns[0], axis=1, inplace=True) # Dropping 'Time'
    # print(data.head(10))
    # data.corr()["COL"]

    if undersampleRatio > 0:
        sampleSize = int(len(data[data['COL'] == 1]) * undersampleRatio)
        if sampleSize < len(data[data['COL'] == 0]):
            print("Undersampling")
            data = concat([data[data["COL"] == 0].sample(sampleSize, random_state=1), data[data["COL"] == True]])
    
    trainX, trainY, testX, testY = splitTrainTest(data, splitRatio=0.8)
    print(f"trainX: {trainX.shape}, trainY: {trainY.shape}, testX: {testX.shape}, testY: {testY.shape}")

    p = NewPredicter()
    trainXpp = p.preProcess(trainX)
    trainYpp = trainY.to_numpy()
    testXpp = p.preProcess(testX)
    testYpp = testY.to_numpy()

    p.fit(trainXpp, trainYpp)
    pred = p.predict(testXpp)
    cm = p.getScore(pred, testYpp)

    return p, data, cm

In [98]:
p, dataRemoveCol, cmRemoveCol = getResultsFromData("../../data/generatedData.csv", pastImportance=4, rowsBeforeCol=4, removeCol=True, undersampleRatio=1)
# dataRemoveCol.corr()["COL"]

Undersampling
Splitting at 598, total rows: 748
trainX: (598, 28), trainY: (598,), testX: (150, 28), testY: (150,)
Scaler is fitted
Total: 150, Collisions: 73
Accuracy: 0.6533333333333333, Precision: 0.6329113924050633, Recall: 0.684931506849315, F1: 0.6578947368421053
Confusion matrix:
[[48 29]
 [23 50]]


c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [51]:
print("Feature   Corr to 'COL'   Min value    Max value")
for c, x, y, z in zip(dataRemoveCol.columns, dataRemoveCol.corr()["COL"], dataRemoveCol.min(), dataRemoveCol.max()):
    print(f"{c.ljust(12)} {str(round(x, 3)).ljust(12)} {str(round(y, 3)).ljust(12)} {str(round(z, 3)).ljust(12)}")


Feature   Corr to 'COL'   Min value    Max value
TTC1         -0.218       0.0          50.0        
TTC2         -0.301       0.0          50.0        
TTC3         -0.378       0.0          50.0        
TTC4         -0.467       0.0          50.0        
DTO1         -0.108       0.0          100.0       
DTO2         -0.223       0.0          100.0       
DTO3         -0.308       0.0          100.0       
DTO4         -0.399       0.0          100.0       
JERK1        -0.07        0.0          65.892      
JERK2        -0.04        0.0          85.116      
JERK3        -0.058       0.0          86.352      
JERK4        -0.095       0.0          33.483      
Speed1       0.214        0.0          32.73       
Speed2       0.212        0.0          34.721      
Speed3       0.202        0.0          32.73       
Speed4       0.185        0.0          34.721      
asX1         0.038        -0.463       0.233       
asX2         -0.049       -0.315       0.254       
asX3         -0

In [87]:
dataRemoveCol[['TTC1', 'TTC2', 'TTC3', 'TTC4', 'DTO1', 'DTO2', 'DTO3', 'DTO4', 'JERK1','JERK2', 'JERK3', 'JERK4', 'Speed1', 'Speed2', 'Speed3', 'Speed4', 'COL']].corr()
# dataRemoveCol[['TTC1', 'TTC2', 'TTC3', 'TTC4']]

,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,JERK2,JERK3,JERK4,Speed1,Speed2,Speed3,Speed4,COL
TTC1,1.000000,0.572853,0.396297,0.286086,0.338424,0.323809,0.330951,0.292208,-0.017977,-0.146863,-0.084863,0.064879,-0.231838,-0.114824,-0.045497,-0.007800,-0.217747
TTC2,0.572853,1.000000,0.560639,0.428378,0.265085,0.387294,0.359175,0.347288,0.085890,-0.055062,-0.076778,0.124697,-0.320184,-0.209518,-0.146671,-0.099175,-0.300520
TTC3,0.396297,0.560639,1.000000,0.574980,0.097355,0.285341,0.390649,0.339234,0.091405,0.071057,0.034610,0.086277,-0.393839,-0.370772,-0.274672,-0.212991,-0.378448
TTC4,0.286086,0.428378,0.574980,1.000000,0.101534,0.194148,0.329302,0.439026,0.096947,0.069894,0.053752,0.141017,-0.341470,-0.349524,-0.346727,-0.285268,-0.467215
DTO1,0.338424,0.265085,0.097355,0.101534,1.000000,0.769329,0.603658,0.511128,-0.144968,-0.125680,-0.058936,-0.059500,0.398637,0.455911,0.457441,0.458051,-0.107781
DTO2,0.323809,0.387294,0.285341,0.194148,0.769329,1.000000,0.770722,0.625268,-0.116848,-0.114748,-0.065790,-0.036706,0.290372,0.364644,0.396546,0.413489,-0.222618
DTO3,0.330951,0.359175,0.390649,0.329302,0.603658,0.770722,1.000000,0.732186,-0.094765,-0.097740,-0.079688,-0.052941,0.151760,0.236185,0.305589,0.340691,-0.308187
DTO4,0.292208,0.347288,0.339234,0.439026,0.511128,0.625268,0.732186,1.000000,-0.049303,-0.043501,-0.017708,-0.017120,0.079263,0.134820,0.193323,0.242183,-0.399014
JERK1,-0.017977,0.085890,0.091405,0.096947,-0.144968,-0.116848,-0.094765,-0.049303,1.000000,0.389400,0.020966,0.082649,-0.234327,-0.228050,-0.204099,-0.186089,-0.069724
JERK2,-0.146863,-0.055062,0.071057,0.069894,-0.125680,-0.114748,-0.097740,-0.043501,0.389400,1.000000,0.662851,0.138306,-0.069479,-0.230282,-0.223485,-0.208238,-0.039798


In [35]:
p, dataNotRemoveCol, cmNotRemoveCol = getResultsFromData("../../data/generatedData.csv", pastImportance=4, rowsBeforeCol=4, removeCol=False, undersampleRatio=1)
# dataNotRemoveCol.corr()["COL"]

Undersampling
Splitting at 942, total rows: 1178
trainX: (942, 28), trainY: (942,), testX: (236, 28), testY: (236,)
Scaler is fitted
Total: 236, Collisions: 119
Accuracy: 0.7372881355932204, Precision: 0.728, Recall: 0.7647058823529411, F1: 0.7459016393442623
Confusion matrix:
[[83 34]
 [28 91]]


c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [37]:
print("Feature, removed, not removed")
for c, x, y, in zip(dataRemoveCol.columns, dataRemoveCol.corr()["COL"], dataNotRemoveCol.corr()["COL"]):
    print(c, round(x, 2), round(y, 2))

Feature, removed, not removed
TTC1 -0.22 -0.3
TTC2 -0.3 -0.38
TTC3 -0.38 -0.45
TTC4 -0.47 -0.44
DTO1 -0.11 -0.2
DTO2 -0.22 -0.28
DTO3 -0.31 -0.36
DTO4 -0.4 -0.42
JERK1 -0.07 -0.07
JERK2 -0.04 -0.02
JERK3 -0.06 0.19
JERK4 -0.09 0.32
Speed1 0.21 0.24
Speed2 0.21 0.22
Speed3 0.2 0.11
Speed4 0.18 -0.06
asX1 0.04 0.04
asX2 -0.05 -0.01
asX3 -0.0 0.04
asX4 -0.05 0.02
asY1 0.01 0.04
asY2 0.07 -0.01
asY3 -0.0 -0.01
asY4 -0.04 -0.05
asZ1 0.03 0.06
asZ2 0.07 0.07
asZ3 0.01 -0.0
asZ4 0.05 -0.01
COL 1.0 1.0


In [42]:
# NOTE DO NOT RUN
# Expected time to run: ~53 minutes

pastImportances = [2,3,4,5,6,7]
rowsBeforeCols = [2,3,4,5,6,7]
removeCols = {False, True}
samplings = [1, 1.5, 2]

allCombos = [(pi, rbc, rc, sr) for pi in pastImportances for rbc in rowsBeforeCols for rc in removeCols for sr in samplings]

for (pi, rbc, rc, sr) in allCombos:
    print(f"\nPast importance: {pi}, sec before col: {rbc}, removeCol: {rc}, sample ratio: {sr}")
    _, _ = getResultsFromData("../../data/generatedData.csv", pastImportance=pi, rowsBeforeCol=rbc, removeCol=rc, undersampleRatio=sr)


Past importance: 2, sec before col: 2, removeCol: False, sample ratio: 1
Undersampling
Splitting at 552, total rows: 690
trainX: (552, 14), trainY: (552,), testX: (138, 14), testY: (138,)
Scaler is fitted


c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Total: 138, Collisions: 73
Accuracy: 0.8623188405797102, Precision: 0.875, Recall: 0.863013698630137, F1: 0.8689655172413793
Confusion matrix:
[[56  9]
 [10 63]]

Past importance: 2, sec before col: 2, removeCol: False, sample ratio: 1.5
Undersampling
Splitting at 689, total rows: 862
trainX: (689, 14), trainY: (689,), testX: (173, 14), testY: (173,)
Scaler is fitted


c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Total: 173, Collisions: 67
Accuracy: 0.8439306358381503, Precision: 0.7857142857142857, Recall: 0.8208955223880597, F1: 0.8029197080291971
Confusion matrix:
[[91 15]
 [12 55]]

Past importance: 2, sec before col: 2, removeCol: False, sample ratio: 2
Undersampling
Splitting at 828, total rows: 1035
trainX: (828, 14), trainY: (828,), testX: (207, 14), testY: (207,)
Scaler is fitted
Total: 207, Collisions: 71
Accuracy: 0.9033816425120773, Precision: 0.84, Recall: 0.8873239436619719, F1: 0.863013698630137
Confusion matrix:
[[124  12]
 [  8  63]]

Past importance: 2, sec before col: 2, removeCol: True, sample ratio: 1
Undersampling
Splitting at 208, total rows: 260
trainX: (208, 14), trainY: (208,), testX: (52, 14), testY: (52,)
Scaler is fitted
Total: 52, Collisions: 31
Accuracy: 0.7115384615384616, Precision: 0.7857142857142857, Recall: 0.7096774193548387, F1: 0.7457627118644068
Confusion matrix:
[[15  6]
 [ 9 22]]

Past importance: 2, sec before col: 2, removeCol: True, sample ratio: 1.5

...

Total: 290, Collisions: 133
Accuracy: 0.6517241379310345, Precision: 0.6095890410958904, Recall: 0.6691729323308271, F1: 0.6379928315412187
Confusion matrix:
[[100  57]
 [ 44  89]]

Past importance: 4, sec before col: 7, removeCol: True, sample ratio: 1.5
Undersampling
Splitting at 1445, total rows: 1807
trainX: (1445, 28), trainY: (1445,), testX: (362, 28), testY: (362,)
Scaler is fitted
Total: 362, Collisions: 123
Accuracy: 0.6988950276243094, Precision: 0.5448717948717948, Recall: 0.6910569105691057, F1: 0.6093189964157706
Confusion matrix:
[[168  71]
 [ 38  85]]

Past importance: 4, sec before col: 7, removeCol: True, sample ratio: 2
Undersampling
Splitting at 1735, total rows: 2169
trainX: (1735, 28), trainY: (1735,), testX: (434, 28), testY: (434,)
Scaler is fitted
Total: 434, Collisions: 136
Accuracy: 0.7142857142857143, Precision: 0.5416666666666666, Recall: 0.5735294117647058, F1: 0.557142857142857
Confusion matrix:
[[232  66]
 [ 58  78]]

Past importance: 5, sec before col: 2

c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Total: 244, Collisions: 125
Accuracy: 0.6352459016393442, Precision: 0.6451612903225806, Recall: 0.64, F1: 0.642570281124498
Confusion matrix:
[[75 44]
 [45 80]]

Past importance: 5, sec before col: 6, removeCol: True, sample ratio: 1.5
Undersampling
Splitting at 1217, total rows: 1522
trainX: (1217, 35), trainY: (1217,), testX: (305, 35), testY: (305,)
Scaler is fitted
Total: 305, Collisions: 122
Accuracy: 0.7311475409836066, Precision: 0.6538461538461539, Recall: 0.6967213114754098, F1: 0.6746031746031745
Confusion matrix:
[[138  45]
 [ 37  85]]

Past importance: 5, sec before col: 6, removeCol: True, sample ratio: 2
Undersampling
Splitting at 1461, total rows: 1827
trainX: (1461, 35), trainY: (1461,), testX: (366, 35), testY: (366,)
Scaler is fitted
Total: 366, Collisions: 108
Accuracy: 0.726775956284153, Precision: 0.5294117647058824, Recall: 0.6666666666666666, F1: 0.5901639344262295
Confusion matrix:
[[194  64]
 [ 36  72]]

Past importance: 5, sec before col: 7, removeCol: False,

c:\Users\Sigurd\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Total: 290, Collisions: 141
Accuracy: 0.6724137931034483, Precision: 0.6575342465753424, Recall: 0.6808510638297872, F1: 0.6689895470383275
Confusion matrix:
[[99 50]
 [45 96]]

Past importance: 5, sec before col: 7, removeCol: True, sample ratio: 1.5
Undersampling
Splitting at 1445, total rows: 1807
trainX: (1445, 35), trainY: (1445,), testX: (362, 35), testY: (362,)
Scaler is fitted
Total: 362, Collisions: 144
Accuracy: 0.712707182320442, Precision: 0.6282051282051282, Recall: 0.6805555555555556, F1: 0.6533333333333333
Confusion matrix:
[[160  58]
 [ 46  98]]

Past importance: 5, sec before col: 7, removeCol: True, sample ratio: 2
Undersampling
Splitting at 1735, total rows: 2169
trainX: (1735, 35), trainY: (1735,), testX: (434, 35), testY: (434,)
Scaler is fitted
Total: 434, Collisions: 143
Accuracy: 0.7396313364055299, Precision: 0.6041666666666666, Recall: 0.6083916083916084, F1: 0.6062717770034843
Confusion matrix:
[[234  57]
 [ 56  87]]

Past importance: 6, sec before col: 2, re

In [16]:
# data = makeDataUsable(read_csv("../../data/testGenerateData.csv"), 6, 4)
dataToUse = data.copy()
dataToUse.drop(columns=data.columns[0], axis=1, inplace=True)
# print(data.head(10))
trainX, trainY, testX, testY = splitTrainTest(dataToUse)
print(f"trainX:{trainX.shape}, trainY:{trainY.shape}, testX:{testX.shape}, testY:{testY.shape}")
# print(trainX.head())
# print(f"trainX: {type(trainX)}, trainY: {type(trainY)}, testX: {type(testX)}, testY: {type(testY)}")


splitting at 215.
trainX:(215, 42), trainY:(215,), testX:(54, 42), testY:(54,)
trainX: <class 'pandas.core.frame.DataFrame'>, trainY: <class 'pandas.core.series.Series'>, testX: <class 'pandas.core.frame.DataFrame'>, testY: <class 'pandas.core.series.Series'>


In [17]:

# p = NewPredicter.loadModel("xgb_2_582-11-16-201")
# print(p.__dict__)

p = NewPredicter()
trainXpp = p.preProcess(trainX)
trainYpp = trainY.to_numpy()
testXpp = p.preProcess(testX)
testYpp = testY.to_numpy()
print(f"trainX: {type(trainXpp)}, trainY: {type(trainYpp)}, testX: {type(testXpp)}, testY: {type(testYpp)}")
# print(trainXpp)
p.fit(trainXpp, trainYpp)

# p.predict([])

Scaler is fitted
trainX: <class 'numpy.ndarray'>, trainY: <class 'numpy.ndarray'>, testX: <class 'numpy.ndarray'>, testY: <class 'numpy.ndarray'>


In [18]:
pred = []
for i in range(len(testX)):
    pred.append(p.predict(testX.iloc[i]))
print(pred)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]


In [19]:
p.getScore(pred, testY)

Total: 54, number of collisions: 14
	TN: 36 	| FP: 4 
	FN: 4 	| TP: 10
Accuracy: 0.85
Precision: 0.71
Recall: 0.71
F1: 0.71


[[36, 4], [4, 10]]